In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"
cohort = "GSE163720"

# Input paths
in_trait_dir = "../../input/GEO/Mesothelioma"
in_cohort_dir = "../../input/GEO/Mesothelioma/GSE163720"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/GSE163720.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/GSE163720.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/GSE163720.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE163720_family.soft.gz', 'GSE163720_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mesothelioma/GSE163720/GSE163720_family.soft.gz
Matrix file: ../../input/GEO/Mesothelioma/GSE163720/GSE163720_series_matrix.txt.gz


Background Information:
!Series_title	"Association of RERG Expression to Female Survival Advantage in Malignant Pleural Mesothelioma I"
!Series_summary	"Sex differences in incidence, prognosis, and treatment response have been described for many cancers. In malignant pleural mesothelioma (MPM), a lethal disease associated with asbestos exposure, men outnumber women 4 to 1, but women consistently live longer than men following surgery-based therapy. This study investigated whether tumor expression of genes associated with estrogen signaling could potentially explain observed survival differences. Two microarray datasets of MPM tumors were analyzed to discover estrogen-related genes associated with survival. A validation cohort of MPM tumors was selected to balance the numbers of men and women and control for competing prognostic influences. The RAS like estrogen regulated growth inhibitor (RERG) gene was identified as the most differentially-expressed estrogen-related gene in these tumo

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any
import numpy as np
import gzip

# 1. Gene Expression Data Availability
# Based on the background information, this seems to be a microarray dataset studying gene expression in MPM tumors
is_gene_available = True  # Microarray data is likely gene expression data

# Read the clinical data from the matrix file
def extract_clinical_data(file_path):
    clinical_data = []
    in_clinical_section = False
    
    with gzip.open(file_path, 'rt') as f:
        for line in f:
            line = line.strip()
            if line.startswith('!Sample_'):
                in_clinical_section = True
                clinical_data.append(line)
            elif in_clinical_section and line.startswith('!'):
                clinical_data.append(line)
            elif line.startswith('#') or line.startswith('^') or line == '':
                in_clinical_section = False
    
    return pd.DataFrame(clinical_data)

# Load clinical data
clinical_data = extract_clinical_data(os.path.join(in_cohort_dir, "GSE163720_series_matrix.txt.gz"))

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, we can see:
# - No explicit trait (mesothelioma) column, but all samples are MPM tumors (from Series_summary)
# - No age data in the sample characteristics dictionary
# - Sex information is available at index 2

# Since all samples are mesothelioma tumors, we'll create a synthetic trait column
trait_row = 0  # We'll use organ: Tumor to denote all samples have mesothelioma
age_row = None  # No age data available
gender_row = 2  # Sex data is available in index 2

# 2.2 Data Type Conversion

# Convert trait function - All samples are mesothelioma tumors
def convert_trait(value):
    # Since all samples are mesothelioma tumors, return 1 for all
    return 1

# Convert age function (not used since age_row is None)
def convert_age(value):
    return None  # We don't have age data to convert

# Convert gender function
def convert_gender(value):
    if value is None:
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert gender to binary (0 for female, 1 for male)
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# All samples are MPM tumors, so trait data is available
is_trait_available = True

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Extract clinical features since trait_row is not None
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical data:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the selected clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{0: [1.0, nan]}
Clinical data saved to ../../output/preprocess/Mesothelioma/clinical_data/GSE163720.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 33295
First 20 gene/probe identifiers:
Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown (7892501, 7892502, etc.), these appear to be probe IDs
# from an Illumina microarray platform rather than human gene symbols.
# These numeric IDs need to be mapped to standard gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001005240,NM_001004195,NM_001005484,BC136848,BC136907', 'BC118988,AL137655', 'NM_001005277,NM_001005221,NM_001005224,NM_001005504,BC137547'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [53049.0, 63015.0, 69091.0, 334129.0, 367659.0], 'RANGE_STOP': [54936.0, 63887.0, 70008.0, 334296.0, 368597.0], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', '---', 'NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005484 // OR4F5 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# Based on the previews:
# - Gene expression data has numeric IDs like '7892501', '7892502', etc.
# - In the gene annotation, the 'ID' column contains similar numeric identifiers
# - The 'gene_assignment' column contains gene symbols and other information

print("Analyzing gene_annotation dataframe for mapping...")
print(f"Gene annotation shape: {gene_annotation.shape}")
print(f"Column names: {gene_annotation.columns.tolist()}")

# 2. Create a gene mapping dataframe
# We need to extract gene symbols from the 'gene_assignment' column
# First check if any rows have gene symbols
print("\nChecking non-empty gene assignments:")
non_empty_assignments = gene_annotation['gene_assignment'].dropna()
print(f"Number of non-empty gene assignments: {len(non_empty_assignments)}")
print("Sample gene assignment:")
print(non_empty_assignments.iloc[0] if len(non_empty_assignments) > 0 else "No non-empty assignments")

# Create the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')
print("\nMapping dataframe created:")
print(f"Shape: {mapping_df.shape}")
print("Sample rows:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print("\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10].tolist())

# Preview the gene expression data
gene_data_preview = preview_df(gene_data)
print("\nPreview of gene expression data:")
print(gene_data_preview)

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Analyzing gene_annotation dataframe for mapping...
Gene annotation shape: (4395073, 12)
Column names: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']

Checking non-empty gene assignments:
Number of non-empty gene assignments: 33297
Sample gene assignment:
---



Mapping dataframe created:
Shape: (33297, 2)
Sample rows:
        ID                                               Gene
0  7896736                                                ---
1  7896738                                                ---
2  7896740  NM_001005240 // OR4F17 // olfactory receptor, ...
3  7896742  ENST00000388975 // SEPT14 // septin 14 // 7p11...
4  7896744  NM_001005277 // OR4F16 // olfactory receptor, ...



Gene expression data after mapping:
Shape: (56391, 131)
First few gene symbols:
['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1']

Preview of gene expression data:
{'GSM4984371': [40.82978590711458, 2.4722163096666665, 3.6812360099999997, 0.4998945246428571, 7.462407054], 'GSM4984372': [42.31433168119358, 2.380555236333333, 3.6982861433333336, 0.49363007257142855, 7.9714218106666666], 'GSM4984373': [41.52395721254568, 2.4463299646666665, 3.750415353333333, 0.5757123258571429, 7.627987897666667], 'GSM4984374': [42.953380502051616, 2.183144652333333, 3.8095943233333336, 0.49244132385714284, 7.634283997666667], 'GSM4984375': [40.92898606758679, 2.400156785, 3.7384230066666664, 0.5014482234285714, 7.467193993333334], 'GSM4984376': [41.35172208334675, 2.39238127, 3.6754141233333333, 0.5450305165714285, 7.8151179903333325], 'GSM4984377': [40.8057277339106, 2.3939998113333334, 3.6602529366666663, 0.5130355922142857, 8.135699602333334], 'GSM4984378': [41.990929820679206, 


Gene expression data saved to ../../output/preprocess/Mesothelioma/gene_data/GSE163720.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Create clinical data from the sample IDs in the gene expression data
# Since all samples are from tumor tissue, they all have mesothelioma (but this is not useful as a trait)
sample_ids = gene_data.columns.tolist()
print(f"Number of samples: {len(sample_ids)}")

# Extract gender information from the original matrix file
gender_data = {}
try:
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Read the matrix file to extract sample characteristics
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Display unique values in each row of clinical data
    characteristics_dict = get_unique_values_by_row(clinical_data)
    print("Sample characteristics dictionary:")
    print(characteristics_dict)
    
    # Try to find gender information in the characteristics
    gender_row = None
    for idx, values in characteristics_dict.items():
        if any('sex:' in str(v).lower() for v in values):
            gender_row = idx
            break
    
    if gender_row is not None:
        # Extract gender data from the row
        for col in clinical_data.columns:
            if col != '!Sample_geo_accession':
                continue
                
            for idx, row in clinical_data.iterrows():
                if idx == gender_row:
                    for i, sample_id in enumerate(clinical_data.iloc[0].values):
                        if i > 0 and sample_id in sample_ids:  # Skip the first column (header)
                            gender_val = clinical_data.iloc[idx, i]
                            if isinstance(gender_val, str) and 'sex:' in gender_val.lower():
                                gender = 0 if 'f' in gender_val.lower() else 1 if 'm' in gender_val.lower() else None
                                gender_data[sample_id] = gender
except Exception as e:
    print(f"Error extracting gender data: {e}")

# Create a clinical dataframe
clinical_df = pd.DataFrame(index=sample_ids)
clinical_df['Mesothelioma'] = 1  # All samples have mesothelioma

# Add gender if available
if gender_data:
    clinical_df['Gender'] = clinical_df.index.map(lambda x: gender_data.get(x))
    print(f"Added gender data for {sum(pd.notna(clinical_df['Gender']))} samples")

print("Clinical data preview:")
print(clinical_df.head())

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data (transpose gene expression data to have samples as rows)
linked_data = pd.concat([clinical_df, gene_data_normalized.T], axis=1)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, 'Mesothelioma')
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait is biased (it will be since all samples are mesothelioma)
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Mesothelioma')

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,  # We do have trait data, it's just that all values are the same
    is_biased=is_trait_biased,  # This will be True since all samples have the same trait value
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from mesothelioma patients only (no controls), making trait biased."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to biased trait variable. Final linked data not saved.")

Shape of gene data after normalization: (20124, 131)


Saved normalized gene data to ../../output/preprocess/Mesothelioma/gene_data/GSE163720.csv
Number of samples: 131


Sample characteristics dictionary:
{0: ['organ: Tumor'], 1: ['compartment: Tissue'], 2: ['Sex: F', 'Sex: M']}
Clinical data preview:
            Mesothelioma
GSM4984371             1
GSM4984372             1
GSM4984373             1
GSM4984374             1
GSM4984375             1
Saved clinical data to ../../output/preprocess/Mesothelioma/clinical_data/GSE163720.csv
Shape of linked data: (131, 20125)


Shape of linked data after handling missing values: (131, 20125)
Quartiles for 'Mesothelioma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Mesothelioma' in this dataset is severely biased.

Dataset validation failed due to biased trait variable. Final linked data not saved.
